In [ ]:
import base64
from openai import OpenAI

client = OpenAI(
)

In [ ]:
import os
import base64
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to encode an image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Function to extract code from an image
extract_code_prompt = """
Extract any developer code from this image. 
If present, return only the code block, with no other explanation.
If no code is present return 'Code NA'.
"""

def extract_code_from_image(image_path):
    base64_image = encode_image(image_path)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": extract_code_prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
    )
    return response.choices[0].message.content

# Function to process a single file
def process_file(file_path):
    try:
        return extract_code_from_image(file_path)
    except Exception as e:
        return f"Error: {e}"

# Function to process the entire folder using ThreadPoolExecutor
def process_folder_parallel(folder_path, max_workers=4):
    results = {}
    file_paths = [
        os.path.join(folder_path, file_name)
        for file_name in sorted(os.listdir(folder_path))
        if os.path.isfile(os.path.join(folder_path, file_name))
    ]

    # Using ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(process_file, file_path): file_path for file_path in file_paths}
        
        for future in as_completed(future_to_file):
            file_path = future_to_file[future]
            file_name = os.path.basename(file_path)
            try:
                results[file_name] = future.result()
            except Exception as e:
                results[file_name] = f"Error: {e}"
    
    return results

# Specify the folder containing the images
folder_path = "./GPTFramesUnique"

# Process the folder in parallel
output_dict = process_folder_parallel(folder_path, max_workers=8)

# Save the results as a dictionary
with open("GPTFramesUnique.json", "w") as output_file:
    json.dump(output_dict, output_file, indent=4)

print("Processing complete. Results saved to 'GPTFramesUnique.json'.")

Processing complete. Results saved to 'GPTFramesUnique.json'.
